In [24]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy.optimize import minimize

In [69]:
def risk_parity_weight(cov_mat, risk_prop, equity_prop):    
    sigma = np.matrix(cov_mat.values)

    def objfun(x):
        tmp = (sigma * np.matrix(x).T).A1
        risk = x * tmp
        var = sum(risk)
        delta_risk = [(risk[i] - risk_prop[i]*var)**2 for i in range(len(risk))]
        return sum(delta_risk)

    x0 = np.ones(sigma.shape[0]) / sigma.shape[0]  
    bnds = tuple((0,None) for x in x0)
    cons = ({'type':'eq', 'fun': lambda x: sum(x) - 1},
            {'type': 'ineq', 'fun': lambda x: equity_prop - x[0]})
    options={'disp':False, 'maxiter':1000, 'ftol':1e-20}

    res = minimize(objfun, x0, bounds=bnds, constraints=cons, method='SLSQP', options=options)

    res['success'] == False
    wts = pd.Series(index=cov_mat.index, data=res['x'])
    return wts / wts.sum() * 1.0

In [95]:
test=pd.read_csv("test.csv")
test.set_index("Date")
cov = test.cov()
target = [0.7, 0.1, 0.2]
equity = 0.8
wts = risk_parity_weight(cov, target, equity)
wts

Stars3     0.646998
Stars4     0.111796
Starts5    0.241206
dtype: float64

In [96]:
var = (wts.T).dot(cov).dot(wts)
wts*(cov.dot(wts))/var

Stars3     0.7
Stars4     0.1
Starts5    0.2
dtype: float64